# 批次

在上一个章节里，我们每次用**一个样本**进行训练。这种方式称为**随机梯度下降**（Stochastic Gradient Descent, SGD）。虽然这种方式简单，但无法充分利用 NumPy 的**向量化并行计算能力**。

实际上，我们可以一次将**多个样本**送入模型进行训练。这种方式称为**批处理**（Batch Processing），是现代深度学习的标准做法。

按照每次使用的样本数量，梯度下降可分为三类：

* **随机梯度下降**（SGD）:每次把一个样本送入迭代。
* **批量梯度下降**（BGD）：每次把所有样本全部送入迭代。实践中很少使用批量梯度下降进行训练，因为需要大量的内存，运算速度会非常缓慢。
* **小批量梯度下降**（Mini-batch GD）：每次把数个样本送入迭代。小批量梯度下降充分地利用了 NumPy（或者其他计算库）的矢量并行运算能力，是目前常规的模型训练方式。

## 收敛

**收敛**（Convergence）指模型训练的过程中，模型通过不断的迭代，其损失值不再显著下降，模型参数趋于稳定。

批量梯度下降每次迭代都可以从全部数据进行学习，因此可以相对保持正确的收敛方向，所以收敛速度较快。

相对应的，随机梯度下降每次迭代只能从一个样本进行学习，因此梯度变化随机性较大，收敛方向容易左右摇摆，所以收敛速度较慢。

小批量梯度下降是折中方案，兼顾效率和稳定性。

## 泛化

**泛化**（Generalization）是评价模型训练效果的一个重要指标，即模型对新数据的推理能力。简单地讲，就是一个在训练集上表现良好的模型，是否可以在测试集上同样有好的表现。

通常来讲，泛化能力来自于对多样化的数据进行学习。因此随机梯度下降的训练方式通常可以带来更好的模型泛化能力；小批量梯度下降次之；批量梯度下降更次之。

泛化能力差一般有两种表现：
* **欠拟合**（Underfitting）：模型在训练集上表现就很差，在测试集上表现也很差。通常是因为训练不足，或者模型过于简单。
* **过拟合**（Overfitting）：模型在训练集上接近优异，但在测试集上表现糟糕。通常是因为模型学习到了过多的、没有代表性的细节。

<div style="border-left: 4px solid #4CAF50; background:#f9f9f9; padding:10px; margin:10px 0;">
<strong>💡 提示：</strong> 批量梯度下降收敛速度快，但是运算慢、泛化能力较差；随机梯度下降泛化能力强，但是收敛较慢；小批量梯度下降则比较均衡，同时具有较好的收敛速度和泛化能力。因此小批量梯度下降是实际工作中最常用的训练方式。
</div>

In [27]:
import numpy as np

## 数据集

### 训练数据：特征、标签

In [28]:
train_features = np.array([[22.5, 72.0],
                           [31.4, 45.0],
                           [19.8, 85.0],
                           [27.6, 63.0]])
train_labels = np.array([[95.0],
                        [210.0],
                        [70.0],
                        [155.0]])

### 测试数据：特征、标签

In [29]:
test_features = np.array([[28.1, 58.0]])
test_labels = np.array([[165.0]])

## 模型

### 参数：权重、偏置

In [30]:
weight = np.ones([1, 2]) / 2
bias = np.zeros(1)

### 推理函数

In [31]:
def forward(x, w, b):
    return x @ w.T + b

### 损失函数（平均平方差）

In [32]:
def mse_loss(p, y):
    return np.mean(np.square(y - p))

### 梯度函数

批处理中，梯度函数将返回和样本个数相同的梯度值。我们将每个梯度除以样本个数，那么在方向函数中将这些梯度累加在一起，就正好构成了一个完整的梯度。

In [33]:
def gradient(p, y):
    return - 2 * (y - p) / len(y)

### 反向函数

反向函数也需要调整，来适应批处理。我们利用 NumPy 的矢量运算来累加梯度。

In [34]:
def backward(x, d, w, b, lr):
    w = w - d.T @ x * lr
    b = b - np.sum(d) * lr
    return w, b

## 训练

### 超参数：学习率

In [35]:
LEARNING_RATE = 0.00001

### 超参数：批大小

作为我们的第二个超参数，**批大小**（Batch Size）定义了每个批次所采用的训练数据的数量。我们将采用小批量梯度下降。

实践中，批大小通常选择2的幂次方（如16, 32, 64, 128...）。这不是数学要求，而是因为这样设置能最大化硬件的运行效率。

In [36]:
BATCH_SIZE = 2

### 迭代

采用了小批量梯度下降后，训练集中的 4 个样本，一轮只需要 2 次迭代。

In [37]:
for i in range(0, len(train_features), BATCH_SIZE):
    # 每次读入一个批次需要的多个训练数据：特征、标签
    features = train_features[i: i + BATCH_SIZE]
    labels = train_labels[i: i + BATCH_SIZE]

    # 同时对多个训练数据进行推理
    predictions = forward(features, weight, bias)
    # 计算多个训练数据的梯度平均值
    delta = gradient(predictions, labels)
    # 用梯度平均值更新一次参数
    weight, bias = backward(features, delta, weight, bias, LEARNING_RATE)

print(f"weight: {weight}")
print(f"bias: {bias}")

weight: [[0.59388172 0.68104165]]
bias: [0.00327249]


## 验证

### 推理

In [38]:
predictions = forward(test_features, weight, bias)
print(f'predictions: {predictions}')

predictions: [[56.19176426]]


### 评估

In [39]:
error = mse_loss(predictions, test_labels)
print(f'loss: {error}')

loss: 11839.232164432306


从验证的结果看，小批量梯度下降一轮迭代的次数变少，但是收敛速度也相应地稍差。